In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import cv2
import os  #dirs 
import numpy as np
from PIL import Image #open image in specific dir
import pickle #alternative to JSON serialization
from google.colab.patches import cv2_imshow


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


______________________________________________________________________________________
# **LOCAL BINARY PATTERN HISTOGRAMS :**
______________________________________________________________________________________

In [ ]:

BASE_DIR = os.path.dirname("/content/drive/MyDrive/CV_Project/face-train.ipynb")
image_dir = os.path.join(BASE_DIR, "TRAINING_IMAGES")

face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")
eye_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_eye.xml")
recognizerLBPH = cv2.face.LBPHFaceRecognizer_create()

current_id = 0
label_ids = {}
y_labels = []
x_train = []

for root, dirs, files in os.walk(image_dir):
  for file in files:
    if file.endswith("png") or file.endswith("jpg"):
      path = os.path.join(root, file)
      label = os.path.basename(root).replace(" ", "-").lower()
      if not label in label_ids:
        label_ids[label] = current_id
        current_id += 1

      id_ = label_ids[label]
      pil_image = Image.open(path).convert("L")  #grayscale
      
      size = (300, 300)
      
      final_image = pil_image 
      final_image = pil_image.resize(size, Image.ANTIALIAS)
      final_image.show()
      
      image_array = np.array(final_image, "uint8")

      faces = face_cascade.detectMultiScale(image_array, scaleFactor=1.1, minNeighbors=4)

      for (x, y, w, h) in faces:
        roi = image_array[y:y+h, x:x+w]
        roi_res = cv2.resize(roi, (200,200), interpolation=cv2.INTER_AREA)

        for (x, y, width, height) in faces:
          eyes = eye_cascade.detectMultiScale(roi[y:(y + height), x:(x + width)], 1.1, 4)
          index = 0
          eye_1 = [None, None, None, None]
          eye_2 = [None, None, None, None]
          for (ex, ey, ew, eh) in eyes:
            if index == 0:
              eye_1 = [ex, ey, ew, eh]
            elif index == 1:
              eye_2 = [ex, ey, ew, eh]
              cv2.rectangle(roi_res[y:(y + height), x:(x + width)], (ex, ey), (ex + ew, ey + eh), (0, 255, 0), 1)
              index = index + 1
            if (eye_1[0] is not None) and (eye_2[0] is not None):
              if eye_1[0] < eye_2[0]:
                left_eye = eye_1
                right_eye = eye_2
              else:
                left_eye = eye_2
                right_eye = eye_1

              left_eye_center = (int(left_eye[0] + (left_eye[2] / 2)), int(left_eye[1] + (left_eye[3] / 2)))
              right_eye_center = (int(right_eye[0] + (right_eye[2] / 2)), int(right_eye[1] + (right_eye[3] / 2)))

              left_eye_x = left_eye_center[0]
              left_eye_y = left_eye_center[1]
              right_eye_x = right_eye_center[0]
              right_eye_y = right_eye_center[1]

              delta_x = right_eye_x - left_eye_x
              delta_y = right_eye_y - left_eye_y

              angle = np.arctan(delta_y / delta_x)

              # Converting radians to degrees
              angle = (angle * 180) / np.pi

              if angle > 5:
                image_center = tuple(np.array(roi_res.shape[1::-1]) / 2)
                rot_mat = cv2.getRotationMatrix2D(image_center, angle, 1.0)
                result = cv2.warpAffine(roi_res, rot_mat, roi_res.shape[1::-1], flags=cv2.INTER_LINEAR)
                roi_res=result
              elif angle < -5:
                image_center = tuple(np.array(roi_res.shape[1::-1]) / 2)
                rot_mat = cv2.getRotationMatrix2D(image_center, angle, 1.0)
                result = cv2.warpAffine(roi_res, rot_mat, roi_res.shape[1::-1], flags=cv2.INTER_LINEAR)
                roi_res=result
              else:
                image_center = tuple(np.array(roi_res.shape[1::-1]) / 2)
                rot_mat = cv2.getRotationMatrix2D(image_center, angle, 1.0)
                result = cv2.warpAffine(roi_res, rot_mat, roi_res.shape[1::-1], flags=cv2.INTER_LINEAR)
                roi_res=result

        roi_res = cv2.resize(roi_res, (200,200), interpolation=cv2.INTER_AREA)
        #roi_res = cv2.GaussianBlur(roi_res,(5,5),1)
        x_train.append(roi_res) 
        y_labels.append(id_)

with open("/content/drive/MyDrive/CV_Project/pickles/face-labels-LBPH.pickle", 'wb') as f: #creating file .pickle and writining in it the serialization of label_ids
  pickle.dump(label_ids, f)

recognizerLBPH.train(x_train, np.array(y_labels))
recognizerLBPH.save("/content/drive/MyDrive/CV_Project/recognizers/face-trainer-LBPH.yml")

______________________________________________________________________________________
# **EIGENFACES :**
______________________________________________________________________________________

In [ ]:
BASE_DIR = os.path.dirname("/content/drive/MyDrive/CV_Project/face-train.ipynb")
image_dir = os.path.join(BASE_DIR, "TRAINING_IMAGES")

face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")
eye_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_eye.xml")
recognizerEigen= cv2.face.EigenFaceRecognizer_create()

current_id = 0
label_ids = {}
y_labels = []
x_train = []

i=0

for root, dirs, files in os.walk(image_dir):
  for file in files:
    if file.endswith("png") or file.endswith("jpg"):
      path = os.path.join(root, file)
      label = os.path.basename(root).replace(" ", "-").lower()
      if not label in label_ids:
        label_ids[label] = current_id
        current_id += 1
      id_ = label_ids[label]
      pil_image = Image.open(path).convert("L")  #grayscale
        
      size = (300, 300)
      final_image = pil_image.resize(size, Image.ANTIALIAS)
      
      image_array = np.array(final_image, "uint8")

      faces = face_cascade.detectMultiScale(image_array, scaleFactor=1.1, minNeighbors=4)

      for (x, y, w, h) in faces:
        roi = image_array[y:y+h, x:x+w]
        roi_res = cv2.resize(roi, (200,200), interpolation=cv2.INTER_AREA)

        for (x, y, width, height) in faces:
          eyes = eye_cascade.detectMultiScale(roi[y:(y + height), x:(x + width)], 1.1, 4)
          index = 0
          eye_1 = [None, None, None, None]
          eye_2 = [None, None, None, None]
          for (ex, ey, ew, eh) in eyes:
            if index == 0:
              eye_1 = [ex, ey, ew, eh]
            elif index == 1:
              eye_2 = [ex, ey, ew, eh]
              cv2.rectangle(roi_res[y:(y + height), x:(x + width)], (ex, ey), (ex + ew, ey + eh), (0, 255, 0), 1)
              index = index + 1
            if (eye_1[0] is not None) and (eye_2[0] is not None):
              if eye_1[0] < eye_2[0]:
                left_eye = eye_1
                right_eye = eye_2
              else:
                left_eye = eye_2
                right_eye = eye_1

              left_eye_center = (int(left_eye[0] + (left_eye[2] / 2)), int(left_eye[1] + (left_eye[3] / 2)))
              right_eye_center = (int(right_eye[0] + (right_eye[2] / 2)), int(right_eye[1] + (right_eye[3] / 2)))

              left_eye_x = left_eye_center[0]
              left_eye_y = left_eye_center[1]
              right_eye_x = right_eye_center[0]
              right_eye_y = right_eye_center[1]

              delta_x = right_eye_x - left_eye_x
              delta_y = right_eye_y - left_eye_y

              angle = np.arctan(delta_y / delta_x)

              # Converting radians to degrees
              angle = (angle * 180) / np.pi

              if angle > 5:
                image_center = tuple(np.array(roi_res.shape[1::-1]) / 2)
                rot_mat = cv2.getRotationMatrix2D(image_center, angle, 1.0)
                result = cv2.warpAffine(roi_res, rot_mat, roi_res.shape[1::-1], flags=cv2.INTER_LINEAR)
                roi_res=result
              elif angle < -5:
                image_center = tuple(np.array(roi_res.shape[1::-1]) / 2)
                rot_mat = cv2.getRotationMatrix2D(image_center, angle, 1.0)
                result = cv2.warpAffine(roi_res, rot_mat, roi_res.shape[1::-1], flags=cv2.INTER_LINEAR)
                roi_res=result
              else:
                image_center = tuple(np.array(roi_res.shape[1::-1]) / 2)
                rot_mat = cv2.getRotationMatrix2D(image_center, angle, 1.0)
                result = cv2.warpAffine(roi_res, rot_mat, roi_res.shape[1::-1], flags=cv2.INTER_LINEAR)
                roi_res=result

        #roi_res=cv2.GaussianBlur(roi_res,(5,5),1)
        roi_res = cv2.resize(roi_res, (200,200), interpolation=cv2.INTER_AREA)
        x_train.append(roi_res) 
        y_labels.append(id_)


with open("/content/drive/MyDrive/CV_Project/pickles/face-labels-Eigen.pickle", 'wb') as f: #creating file .pickle and writining in it the serialization of label_ids
  pickle.dump(label_ids, f)            


recognizerEigen.train(x_train, np.array(y_labels))
recognizerEigen.save("/content/drive/MyDrive/CV_Project/recognizers/face-trainer-Eigen.yml")

______________________________________________________________________________________
# **FISHERFACES :**
______________________________________________________________________________________

In [ ]:
BASE_DIR = os.path.dirname("/content/drive/MyDrive/CV_Project/face-train.ipynb")
image_dir = os.path.join(BASE_DIR, "TRAINING_IMAGES")

face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")
eye_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_eye.xml")
recognizerFisher= cv2.face.FisherFaceRecognizer_create()

current_id = 0
label_ids = {}
y_labels = []
x_train = []

for root, dirs, files in os.walk(image_dir):
  for file in files:
    if file.endswith("png") or file.endswith("jpg"):
      path = os.path.join(root, file)
      label = os.path.basename(root).replace(" ", "-").lower()
      if not label in label_ids:
        label_ids[label] = current_id
        current_id += 1
      id_ = label_ids[label]
      pil_image = Image.open(path).convert("L")  # grayscale
      
      size = (300, 300)
      final_image = pil_image.resize(size, Image.ANTIALIAS)
      
      image_array = np.array(final_image, "uint8")

      faces = face_cascade.detectMultiScale(image_array, scaleFactor=1.1, minNeighbors=4)

      for (x, y, w, h) in faces:
        roi = image_array[y:y+h, x:x+w]
        roi_res = cv2.resize(roi, (200,200), interpolation=cv2.INTER_AREA).flatten()

        for (x, y, width, height) in faces:
          eyes = eye_cascade.detectMultiScale(roi[y:(y + height), x:(x + width)], 1.1, 4)
          index = 0
          eye_1 = [None, None, None, None]
          eye_2 = [None, None, None, None]
          for (ex, ey, ew, eh) in eyes:
            if index == 0:
              eye_1 = [ex, ey, ew, eh]
            elif index == 1:
              eye_2 = [ex, ey, ew, eh]
              cv2.rectangle(roi_res[y:(y + height), x:(x + width)], (ex, ey),
                          (ex + ew, ey + eh), (0, 255, 0), 1)
              index = index + 1
            if (eye_1[0] is not None) and (eye_2[0] is not None):
              if eye_1[0] < eye_2[0]:
                left_eye = eye_1
                right_eye = eye_2
              else:
                left_eye = eye_2
                right_eye = eye_1
              left_eye_center = (
                int(left_eye[0] + (left_eye[2] / 2)),
                int(left_eye[1] + (left_eye[3] / 2)))

              right_eye_center = (
                int(right_eye[0] + (right_eye[2] / 2)),
                int(right_eye[1] + (right_eye[3] / 2)))

              left_eye_x = left_eye_center[0]
              left_eye_y = left_eye_center[1]
              right_eye_x = right_eye_center[0]
              right_eye_y = right_eye_center[1]

              delta_x = right_eye_x - left_eye_x
              delta_y = right_eye_y - left_eye_y

              angle = np.arctan(delta_y / delta_x)

              # Converting radians to degrees
              angle = (angle * 180) / np.pi

              if angle > 5:
                image_center = tuple(np.array(roi_res.shape[1::-1]) / 2)
                rot_mat = cv2.getRotationMatrix2D(image_center, angle, 1.0)
                result = cv2.warpAffine(roi_res, rot_mat, roi_res.shape[1::-1], flags=cv2.INTER_LINEAR)
                roi_res=result
              elif angle < -5:
                image_center = tuple(np.array(roi_res.shape[1::-1]) / 2)
                rot_mat = cv2.getRotationMatrix2D(image_center, angle, 1.0)
                result = cv2.warpAffine(roi_res, rot_mat, roi_res.shape[1::-1], flags=cv2.INTER_LINEAR)
                roi_res=result
              else:
                image_center = tuple(np.array(roi_res.shape[1::-1]) / 2)
                rot_mat = cv2.getRotationMatrix2D(image_center, angle, 1.0)
                result = cv2.warpAffine(roi_res, rot_mat, roi_res.shape[1::-1], flags=cv2.INTER_LINEAR)
                roi_res=result

        #roi_res=cv2.GaussianBlur(roi_res,(5,5),1)
        roi_res = cv2.resize(roi, (200,200), interpolation=cv2.INTER_AREA)
        x_train.append(roi_res) 
        y_labels.append(id_)


with open("/content/drive/MyDrive/CV_Project/pickles/face-labels-Fisher.pickle", 'wb') as f: #creating file .pickle and writining in it the serialization of label_ids 
  pickle.dump(label_ids, f)           


recognizerFisher.train(x_train, np.array(y_labels))
recognizerFisher.save("/content/drive/MyDrive/CV_Project/recognizers/face-trainer-Fisher.yml")